# First Attempt at DeepNN for tumor subtype classification

This will need a lot of work so bare with me :3

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rc

from tqdm import tqdm

from custom_functions import store_json, load_json

In [ ]:
# Constants
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

RANDOM_SEED = 42
EPOCHS = 200
BATCH_SIZE = 16

HAPPY_COLORS_PALETTE =\
["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

## Data

In [ ]:
dfr = pd.read_csv("./data/Train_call.txt", sep='\t')
dfr_sols = pd.read_csv("./data/Train_clinical.txt", sep='\t')

df_instances = dfr.T

df_instances["Subgroup"] = None if type(dfr_sols.Subgroup.values[-1]) != int else 0

for arr_name in dfr_sols["Sample"]:
    df_instances.at[arr_name, "Subgroup"] =\
     dfr_sols[dfr_sols["Sample"] == arr_name]["Subgroup"].values[0]

if None in df_instances.Subgroup:
    df_instances.Subgroup.replace({None: "dummy"}, inplace=True)

In [ ]:
df_instances.Subgroup = df_instances.Subgroup.astype('|S')

# df_instances.drop([
#     "Chromosome",
#     "Start",
#     "End",
#     "Nclone",
# ]).to_csv("./data/fastai_superset.csv")

df = df_instances.drop([
    "Chromosome",
    "Start",
    "End",
    "Nclone",
])

for col in df.columns:
    try:
        if "Subgroup" in col:
            continue
    except:
        pass
    df[col] = df[col].apply(lambda x:x+2)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,2825,2826,2827,2828,2829,2830,2831,2832,2833,Subgroup
Array.129,2,2,2,2,2,2,2,2,2,2,...,4,4,4,2,3,3,3,3,3,b'HER2+'
Array.34,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,3,3,3,3,b'HR+'
Array.67,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,3,3,3,3,b'HR+'
Array.24,2,2,2,2,2,2,2,1,2,2,...,2,2,2,2,2,2,2,2,2,b'Triple Neg'
Array.22,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,3,3,3,3,b'Triple Neg'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,2,2,2,2,2,2,2,2,2,2,...,3,2,3,3,3,3,3,3,3,b'HER2+'
Array.123,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,3,3,3,3,b'HR+'
Array.100,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,3,3,3,3,b'HR+'
Array.134,1,1,1,1,1,1,1,1,1,1,...,3,3,3,3,3,3,3,3,3,b'HR+'


## DeepNN

In [ ]:
%%capture
!pip install fastai

In [ ]:
from fastai.tabular.all import *

/root/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
path = Path("./data/")

In [ ]:
splits = IndexSplitter(list(range(80,100)))(range_of(df)); splits

((#80) [0,1,2,3,4,5,6,7,8,9...], (#20) [80,81,82,83,84,85,86,87,88,89...])

In [ ]:
target = 'Subgroup'
cat_names = [] # Categorical
cont_names = [] # Continous
procs = [FillMissing, Categorify, Normalize]

to = TabularPandas(df, procs=procs, y_names=target, splits=splits)

In [ ]:
dls = to.dataloaders(bs=512)

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

learn.lr_find()

ZeroDivisionError: float division by zero

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c915e4f9-60c2-40b5-a522-8a90cb3fd50a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>